In [18]:
import numpy as np
from numpy import array
from numpy.polynomial.polynomial import polyfit
from tqdm import tqdm

from xlstm_mixer.lit.data import Task, TSLibDataModule

In [19]:
def hurst(ts: np.ndarray) -> np.ndarray:
    """
    From: https://github.com/RyanWangZf/Hurst-exponent-R-S-analysis-/blob/master/Hurst.py
    """
    N = ts.shape[0]
    if N < 20:
        raise ValueError(
            "Time series is too short! input series ought to have at least 20 samples!"
        )

    max_k = N // 2
    R_S_dict = []
    for k in tqdm(range(10, max_k + 1)):
        R, S = 0, 0
        # split ts into subsets
        subset_list = [ts[i : i + k, :] for i in range(0, N, k)]
        if np.mod(N, k) > 0:
            subset_list.pop()

        # calc mean of every subset
        for subset in subset_list:
            cumsum_list = np.cumsum(subset - subset.mean(axis=0), axis=0)
            R += np.max(cumsum_list, axis=0) - np.min(cumsum_list, axis=0)
            S += np.std(subset, axis=0)
        R_S_dict.append((R / len(subset_list), S / len(subset_list), k))

    log_R_S = []
    log_n = []

    for R, S, n in R_S_dict:
        log_n.append(np.log(n))
        R_S = (R + np.spacing(1)) / (S + np.spacing(1))
        log_R_S.append(np.log(R_S))

    return polyfit(log_n, log_R_S, 1)[-1, :]


# Try:
ts = np.random.rand(1_000, 2)
# ts[:, 0] = np.cos(np.arange(1_000) % 2.0)
# ts[:, 0] = np.cos(np.arange(1_000) / 1_000.0)
hurst(ts)

100%|██████████| 491/491 [00:00<00:00, 1246.46it/s]


array([0.50337593, 0.49924713])

In [20]:
datasets = [
    "Weather",
    "ETTh1",
    "ETTh2",
    "ETTm1",
    "ETTm2",
    "Electricity",
    "Exchange",
    "Solar",
    "Traffic",
    "Ili",
]
H_values = {}

In [ ]:
for dataset in datasets:
    if dataset in H_values:
        print(f"Skipping {dataset} dataset as it has already been calculated")
        continue

    print(f"Calculating Hurst Exponent for {dataset} dataset")

    data_module = TSLibDataModule(
        task=Task.LONG_TERM_FORECAST, dataset_name=dataset, num_workers=0
    )
    data_module.embed = "timeF"
    data_module.freq = "W"
    data_module.seq_len = 336
    data_module.label_len = 0
    data_module.pred_len = 720
    data_module.features = "MS"
    data_module.target = "OT"
    data_module.seasonal_patterns = None

    if dataset == "Ili":
        data_module.seq_len = 20
        data_module.pred_len = 30

    data_module.setup("fit")
    data = data_module.train_dataset._data_all  # includes train, val, and test

    H_values_variates = hurst(data)
    H_values[dataset] = H_values_variates

Calculating Hurst Exponent for Exchange dataset
train 4256
val 41


100%|██████████| 3785/3785 [00:04<00:00, 799.87it/s] 


In [21]:
H_values

{}

In [22]:
H_values = {
    "Weather": array(
        [
            0.80142176,
            1.01528083,
            1.00631775,
            0.90463039,
            0.94611491,
            1.01963684,
            0.90687494,
            0.97728952,
            0.90456967,
            0.90463628,
            0.95788105,
            0.33302144,
            0.81261352,
            0.8552803,
            0.63193458,
            0.71179446,
            0.99884365,
            1.00576408,
            0.99772455,
            1.03830606,
            0.50183524,
        ]
    ),
    "ETTh1": array(
        [
            0.9834697,
            1.04147944,
            0.96823539,
            1.07006409,
            1.0044969,
            0.94267486,
            1.09817671,
        ]
    ),
    "ETTh2": array(
        [
            0.90565816,
            1.04367403,
            0.99165408,
            1.06774359,
            0.9946176,
            0.94996812,
            1.13327527,
        ]
    ),
    "ETTm1": array(
        [
            0.97795727,
            1.03778551,
            0.96186627,
            1.06656588,
            0.99844366,
            0.94484742,
            1.09465053,
        ]
    ),
    "ETTm2": array(
        [
            0.90802315,
            1.04222523,
            0.9918761,
            1.06492962,
            0.99685033,
            0.95316695,
            1.12737572,
        ]
    ),
    "Electricity": array(
        [
            1.07827311,
            1.05093915,
            1.0192512,
            1.19688509,
            1.13573192,
            1.20999045,
            0.96774448,
            0.99494064,
            1.07735322,
            1.02977754,
            1.1772559,
            0.6639689,
            1.05976412,
            1.00832099,
            1.04718738,
            0.93513461,
            1.00087996,
            1.05888557,
            1.09027116,
            0.90033111,
            0.88597373,
            1.10136505,
            1.17064897,
            1.20212543,
            1.15600716,
            0.77216082,
            1.14630534,
            0.89518123,
            1.04208161,
            0.77132208,
            0.97676416,
            0.99139871,
            1.2407155,
            1.03368901,
            1.17595976,
            1.08778418,
            0.85085242,
            1.18821644,
            1.03479948,
            0.92589939,
            1.13151704,
            1.08594871,
            0.98746091,
            1.14948931,
            1.05985906,
            1.06732466,
            1.1762417,
            1.06147799,
            0.88454397,
            1.00323205,
            1.02748208,
            1.13896447,
            1.13266625,
            1.17325172,
            1.05806874,
            0.97252316,
            1.02691578,
            1.240883,
            1.04691143,
            1.09700455,
            1.20323801,
            1.21984835,
            1.22326762,
            1.24958304,
            1.03038406,
            1.09503563,
            1.16361528,
            1.18654964,
            0.97755418,
            1.14859282,
            1.01088326,
            0.96901328,
            1.06726534,
            0.96480921,
            1.1754059,
            1.00678659,
            1.08278658,
            1.00820885,
            0.88412794,
            1.08307782,
            1.07945485,
            0.60039279,
            0.97684693,
            0.75452673,
            0.84950347,
            1.04527532,
            1.01666353,
            0.88297441,
            1.0041545,
            1.10449064,
            1.20841398,
            1.16954768,
            1.11190933,
            0.98711396,
            1.14381016,
            1.20697254,
            0.91420456,
            1.18178147,
            1.03196359,
            0.88462327,
            0.89872245,
            0.92932795,
            1.13785949,
            0.76745566,
            1.0643018,
            0.79506793,
            0.80165366,
            0.55473008,
            0.89552396,
            0.93620751,
            1.07399828,
            1.13064795,
            1.09856833,
            0.70934071,
            1.13394486,
            1.07201291,
            1.04853703,
            1.10901505,
            1.06730213,
            1.17244404,
            1.04479728,
            1.1142243,
            1.08887583,
            1.08938861,
            0.91015752,
            1.09981713,
            0.73742737,
            0.97671775,
            1.053829,
            0.85335152,
            0.97106888,
            0.96891894,
            0.90270039,
            0.87236845,
            1.02145153,
            1.12692355,
            0.87554411,
            0.97998517,
            0.92231151,
            0.84810531,
            1.11284065,
            0.94364951,
            1.04015865,
            0.84805795,
            1.14454227,
            0.89457597,
            1.24882082,
            1.19946467,
            1.02960804,
            1.17336291,
            1.15669045,
            1.08037743,
            1.19784864,
            1.21169246,
            0.91828944,
            1.14856346,
            1.02237446,
            1.05864728,
            1.22350377,
            1.14788579,
            1.13633217,
            1.1461777,
            1.15479229,
            1.22858087,
            1.00592023,
            1.06605775,
            1.0416557,
            1.18190862,
            1.10579397,
            1.13110781,
            1.10825569,
            1.11807512,
            1.15529063,
            1.15701943,
            1.14673458,
            1.24399805,
            1.20445746,
            1.14737501,
            1.21468902,
            1.12793434,
            1.1641595,
            1.20655249,
            1.36401399,
            1.2151568,
            1.15131253,
            1.14740282,
            1.20973846,
            1.16653077,
            1.14814099,
            1.01848836,
            1.04556668,
            1.19623723,
            1.14531497,
            0.89282002,
            1.15109797,
            1.0344036,
            1.02255054,
            1.1514546,
            0.93819106,
            1.05442156,
            0.98980565,
            1.09860147,
            1.12763688,
            1.17159272,
            1.04574053,
            1.11631904,
            1.01882406,
            1.07209605,
            1.09183788,
            0.964025,
            1.00790467,
            0.94857518,
            1.05286611,
            1.0031588,
            1.03916527,
            1.05478648,
            1.20849399,
            1.18140968,
            0.95141344,
            1.07495765,
            0.93185861,
            1.02016912,
            0.98311446,
            1.03304426,
            0.94284643,
            1.10919818,
            1.06865227,
            0.92381328,
            1.14734642,
            1.08985966,
            1.15098192,
            1.07479284,
            1.09502034,
            1.14809574,
            1.11296006,
            1.00757649,
            1.08228418,
            1.05767249,
            1.01320137,
            1.12979028,
            1.13769097,
            1.16670582,
            1.16329789,
            1.10264398,
            1.06229786,
            1.1925148,
            0.98629563,
            0.90462549,
            0.95612967,
            0.97532755,
            0.99544516,
            0.96188235,
            1.10736945,
            0.91759896,
            1.04831884,
            1.03692602,
            0.98795064,
            1.1335765,
            0.92618318,
            0.97389357,
            0.93872468,
            0.98213874,
            1.04399149,
            1.09162382,
            1.04206068,
            1.04339988,
            0.95495509,
            1.15340296,
            1.20199421,
            1.22308802,
            1.20573453,
            1.07912639,
            1.16694105,
            1.20707049,
            1.11203615,
            1.14575526,
            1.12652357,
            1.10309772,
            1.28274993,
            1.16366106,
            1.11959525,
            1.11307113,
            1.12649558,
            1.14825398,
            1.0648568,
            1.0078573,
            0.99518981,
            1.09090224,
            0.88540772,
            0.96417759,
            1.17814402,
            1.03778104,
            1.0403486,
            1.06436984,
            1.20413382,
            0.98534566,
            1.0443247,
            1.04121434,
            1.19432562,
            0.74170745,
            1.17817772,
            1.06834393,
            1.11436037,
            1.15043348,
            1.14809044,
            1.04719732,
            1.12862003,
            1.13701612,
            1.027553,
            1.17760969,
            1.19077407,
            1.16034077,
            1.13089098,
            1.26051271,
            1.23135308,
            1.21911559,
            1.14632747,
            1.00933272,
            1.09075481,
            0.62482919,
            1.10298988,
        ]
    ),
    "Solar": array(
        [
            0.80610894,
            0.71396586,
            0.75384838,
            0.75383172,
            0.79406518,
            0.65905655,
            0.73392543,
            0.75054598,
            0.75460867,
            0.77626804,
            0.788481,
            0.64436309,
            0.74534205,
            0.78127261,
            0.67924508,
            0.78044531,
            0.75893677,
            0.75689887,
            0.7716279,
            0.77957279,
            0.74294761,
            0.74857764,
            0.78593185,
            0.64559763,
            0.74900845,
            0.76742908,
            0.77739051,
            0.71552811,
            0.67079691,
            0.77490289,
            0.79611633,
            0.64215351,
            0.7475577,
            0.7487473,
            0.77093555,
            0.64129267,
            0.75604754,
            0.7557071,
            0.75663239,
            0.77482049,
            0.65468376,
            0.77384298,
            0.73662367,
            0.77460397,
            0.80966912,
            0.79740299,
            0.74705425,
            0.72585809,
            0.71802959,
            0.75920859,
            0.7554532,
            0.7706489,
            0.72690395,
            0.75463364,
            0.7342162,
            0.74037339,
            0.74776311,
            0.76792399,
            0.73342981,
            0.75164129,
            0.75219067,
            0.73664816,
            0.76832416,
            0.75292928,
            0.7289751,
            0.75164795,
            0.7519213,
            0.72186713,
            0.82167031,
            0.6323392,
            0.6382651,
            0.72587553,
            0.72275406,
            0.78859439,
            0.74070545,
            0.75363308,
            0.68790479,
            0.77398987,
            0.73757632,
            0.72907791,
            0.6415825,
            0.77034441,
            0.7342272,
            0.75149801,
            0.76307506,
            0.74538665,
            0.75932104,
            0.74569881,
            0.7684374,
            0.75160754,
            0.72379077,
            0.72507747,
            0.75283265,
            0.73703668,
            0.75788828,
            0.72611191,
            0.79822303,
            0.74609856,
            0.77297707,
            0.72294478,
            0.75409026,
            0.75440151,
            0.77706044,
            0.76942621,
            0.68986147,
            0.63143822,
            0.76603366,
            0.74737601,
            0.74158014,
            0.74657201,
            0.76481259,
            0.72818252,
            0.6472674,
            0.76334688,
            0.72820907,
            0.64153636,
            0.73052496,
            0.71326261,
            0.75798363,
            0.7421945,
            0.7502932,
            0.80375288,
            0.73228128,
            0.74842513,
            0.64734646,
            0.6579658,
            0.67931564,
            0.73061324,
            0.7535093,
            0.63637553,
            0.64766147,
            0.780004,
            0.77326567,
            0.71421045,
            0.75472725,
            0.78176738,
            0.68532971,
        ]
    ),
    "Traffic": array(
        [
            0.67608693,
            0.61029632,
            0.77407012,
            0.62726665,
            0.55098425,
            0.56733171,
            0.51959513,
            0.71642865,
            0.7329147,
            0.67925066,
            0.66346247,
            0.72810692,
            0.68861857,
            0.59408607,
            0.7310276,
            0.74398925,
            0.93972326,
            0.79844938,
            0.61888189,
            0.88240525,
            0.59491975,
            0.7353525,
            0.61978621,
            0.78384467,
            0.77120073,
            0.74687926,
            0.71332131,
            0.92063801,
            0.90571668,
            0.82070652,
            0.51079087,
            0.86886799,
            0.66699378,
            0.83777566,
            0.66228313,
            0.83418713,
            0.73202169,
            0.81145423,
            0.88098,
            0.16160546,
            0.62574697,
            0.57105365,
            0.78358968,
            0.69807416,
            1.21307285,
            0.71243531,
            0.79483651,
            0.68723278,
            0.77328197,
            0.57625812,
            0.58393276,
            0.68035306,
            0.70745265,
            0.89404727,
            0.65683008,
            0.93144933,
            0.74215034,
            0.88433438,
            0.52941667,
            0.75019187,
            0.76834459,
            0.63467977,
            0.98870677,
            0.83849054,
            0.62154336,
            0.6117037,
            0.72407506,
            0.54669286,
            0.75680911,
            1.00506189,
            0.72954527,
            0.64073866,
            0.76929934,
            0.9389163,
            0.6797537,
            0.54131525,
            0.54556045,
            0.56522055,
            0.78541576,
            0.5732568,
            0.95309856,
            0.76704175,
            0.66643202,
            0.89187751,
            0.68512738,
            0.80379743,
            0.66408537,
            0.79816566,
            0.8023189,
            0.6674697,
            0.79831081,
            0.94579684,
            0.77006208,
            0.63495332,
            0.7046123,
            0.81228543,
            0.58570714,
            0.55091028,
            0.57639075,
            0.58691165,
            0.49762451,
            0.7812884,
            0.82835743,
            0.75000791,
            0.77590956,
            0.9419717,
            0.90727346,
            0.5361257,
            0.86359157,
            0.93982107,
            1.06619436,
            0.85556326,
            0.74374911,
            0.86343839,
            0.57960259,
            0.92879041,
            0.71771773,
            1.10849541,
            0.75500072,
            0.68315117,
            0.57927355,
            0.64189668,
            0.50084461,
            0.6598669,
            0.69268594,
            0.76598491,
            0.79777854,
            1.00418239,
            0.73248234,
            0.66361443,
            0.7388052,
            0.69482664,
            0.80525169,
            0.63794075,
            0.94187083,
            0.63320946,
            0.88635396,
            0.4964284,
            0.79105956,
            0.58900136,
            0.87598412,
            0.74161557,
            0.98770851,
            0.61256065,
            0.79081462,
            0.93847089,
            0.54444933,
            0.6704805,
            0.72648751,
            0.66790635,
            0.70955251,
            0.74255282,
            0.58216209,
            0.65944289,
            0.54729478,
            0.70682266,
            0.85713979,
            0.9741862,
            0.54910772,
            0.71163518,
            0.64642503,
            0.75755518,
            0.62906092,
            0.85822948,
            0.85550632,
            0.73190286,
            0.82879274,
            0.76329843,
            0.90983119,
            0.67187196,
            0.67360927,
            0.7036784,
            0.55152007,
            0.93929698,
            0.93485726,
            0.6027221,
            0.74785039,
            0.74401722,
            0.72201051,
            0.82243982,
            0.86403811,
            0.70841697,
            0.91228697,
            0.77517838,
            0.77373524,
            0.67115619,
            0.57552843,
            0.71966521,
            0.64691068,
            0.76518118,
            0.73079837,
            0.65991055,
            0.60444567,
            0.9024503,
            0.78960093,
            0.6158368,
            0.71345318,
            0.54461354,
            0.97168698,
            0.90643217,
            0.54311992,
            0.63506284,
            0.65074161,
            0.60418634,
            0.7816953,
            0.85546448,
            0.62137668,
            0.92477722,
            1.10531563,
            0.61398053,
            0.77205861,
            0.69175499,
            0.65962783,
            0.60695822,
            0.81726866,
            0.83098092,
            0.69246336,
            0.81973405,
            0.88902658,
            0.86837077,
            0.8682233,
            0.67100621,
            0.73020619,
            0.90183777,
            0.79324949,
            0.8099582,
            0.89638607,
            0.66416171,
            0.71484292,
            0.84943887,
            0.68607525,
            0.69837299,
            0.77371596,
            0.92544993,
            1.10966638,
            0.86458544,
            0.70735996,
            0.74571586,
            0.65791546,
            0.72949457,
            0.62063498,
            0.67989264,
            0.81358113,
            0.95432047,
            0.72256071,
            0.6841935,
            0.54332224,
            0.70146933,
            1.16099245,
            0.60336362,
            0.72983737,
            0.76835221,
            0.66107613,
            0.56989762,
            0.99839513,
            0.63909773,
            0.83114502,
            0.74339329,
            1.05209716,
            0.5970088,
            0.64394888,
            0.59467711,
            0.61057757,
            0.73808831,
            0.72875356,
            0.79778394,
            0.7902584,
            0.1637379,
            0.63989136,
            0.73504839,
            0.7532901,
            0.63764269,
            0.66729376,
            0.79001292,
            0.56009131,
            0.57304294,
            0.75386164,
            0.7472629,
            0.5330735,
            0.7738159,
            0.84363098,
            0.84621898,
            0.74315962,
            0.81860721,
            0.49194352,
            1.2028008,
            0.96159044,
            0.75480896,
            0.68610107,
            0.75175178,
            0.54483814,
            0.64201041,
            0.47967826,
            0.84354258,
            0.75364837,
            0.76127013,
            0.77034141,
            0.74536815,
            0.81587889,
            0.74929543,
            0.50861255,
            0.84314488,
            0.84215611,
            0.60626553,
            0.66629017,
            0.87214248,
            0.58297381,
            0.92575932,
            0.74666354,
            1.03615365,
            0.72617905,
            0.82373121,
            0.64278177,
            0.76549357,
            0.84927187,
            0.72021812,
            0.74633122,
            0.99886886,
            0.88912961,
            0.6804135,
            0.62416433,
            0.7385246,
            0.66026803,
            0.81306137,
            0.66896514,
            0.91534616,
            0.7295005,
            1.03306194,
            0.7325226,
            0.80093977,
            0.69639646,
            0.66479872,
            0.82795394,
            0.86578126,
            0.66666924,
            0.60625371,
            0.85824238,
            0.65335342,
            0.79327655,
            0.53069414,
            0.65008456,
            0.63893741,
            0.58273561,
            0.58688845,
            0.61752897,
            0.74834859,
            0.61594224,
            0.69810775,
            1.01111845,
            0.65829636,
            0.5326057,
            0.88493977,
            0.87018805,
            1.05306159,
            0.90777658,
            0.68441365,
            0.9979461,
            0.65085402,
            0.80638931,
            0.6117631,
            0.6597329,
            0.4970035,
            0.74217456,
            0.84388246,
            0.82427466,
            0.94832732,
            1.01902566,
            0.86660867,
            0.57189767,
            0.78449957,
            0.62071615,
            0.98524392,
            0.54758735,
            0.56959661,
            0.62433888,
            0.53980257,
            0.692524,
            1.0716313,
            0.6140506,
            0.79444442,
            0.67268164,
            0.68970956,
            0.5403217,
            0.75991551,
            0.62756011,
            0.71672437,
            0.47645466,
            0.66019484,
            1.0157249,
            0.5773474,
            0.98622415,
            0.92758795,
            0.69134971,
            0.78662528,
            0.78151386,
            0.58765226,
            0.82392622,
            0.5855126,
            0.62079949,
            0.76097569,
            0.71878125,
            0.73371851,
            0.63122577,
            0.60917804,
            0.62348782,
            0.73769845,
            0.78727185,
            0.50490006,
            0.76000271,
            0.58983273,
            0.49974801,
            0.61480631,
            0.6931115,
            0.51162696,
            0.61374806,
            0.89317838,
            0.69685875,
            0.78563742,
            0.84366518,
            0.7710634,
            0.65233457,
            0.55485877,
            0.63814935,
            0.62874109,
            0.64879381,
            1.05662391,
            0.74115867,
            0.63105043,
            0.87705837,
            0.73064866,
            0.6041485,
            0.6844222,
            0.7228143,
            0.776401,
            0.67903175,
            0.80448285,
            0.63969266,
            0.66342438,
            0.60724734,
            0.82778211,
            0.73366318,
            0.65900849,
            0.79958757,
            0.80218257,
            0.51905086,
            1.0107395,
            1.0753532,
            0.9532768,
            0.8885583,
            0.77389825,
            0.78498118,
            0.74742995,
            0.72500357,
            0.6331178,
            0.62607674,
            0.52868577,
            0.70355933,
            1.03962247,
            0.6836799,
            0.59387639,
            0.72940038,
            0.64804493,
            0.62291294,
            0.65855198,
            0.6321718,
            0.89977311,
            0.76084772,
            0.7347557,
            0.67403375,
            0.79696796,
            0.65268703,
            0.67471882,
            1.09068728,
            0.60849254,
            0.53366924,
            0.81159469,
            0.72580279,
            0.84213589,
            0.80668871,
            0.67176299,
            0.6989123,
            0.84343862,
            0.79423672,
            0.61932137,
            0.63475757,
            0.73100433,
            0.91926174,
            0.63470445,
            0.78888952,
            0.55603133,
            0.82957256,
            0.61833518,
            0.78094005,
            0.66776332,
            0.54332046,
            0.92010805,
            1.10088401,
            0.6177026,
            0.61014333,
            0.63530077,
            1.0286895,
            1.08487719,
            0.66349825,
            0.62766205,
            0.63628742,
            0.77609758,
            0.57033471,
            0.99986474,
            0.75052851,
            0.74006558,
            1.15937224,
            0.82130708,
            0.92676705,
            0.54606389,
            0.61444567,
            0.78151006,
            0.93887808,
            0.94140453,
            0.65865761,
            0.73509008,
            0.8816045,
            0.96137725,
            0.93466744,
            0.62258142,
            0.73409838,
            0.54019153,
            0.73292094,
            0.64532422,
            0.69109457,
            0.70758892,
            0.77354427,
            0.71697795,
            0.76957083,
            0.70952423,
            0.64889134,
            0.61116441,
            0.79431466,
            0.85556042,
            0.62838495,
            0.82701042,
            0.64730287,
            0.85873141,
            0.89503524,
            0.89690596,
            0.87961637,
            0.82152372,
            0.60892511,
            0.67309174,
            0.69883434,
            0.56005774,
            0.65890702,
            0.63439525,
            0.83555325,
            0.63900928,
            0.79053559,
            1.01066914,
            0.9908892,
            0.65364549,
            0.62448686,
            0.63596401,
            0.55748306,
            0.61700898,
            0.56325359,
            0.91321479,
            0.73926217,
            1.06793074,
            0.67661894,
            1.10234496,
            0.51108681,
            1.08195743,
            0.89492209,
            0.98568602,
            0.57601513,
            0.758771,
            0.61447592,
            0.97253738,
            0.73850609,
            0.80981403,
            0.57040314,
            0.77336647,
            0.70819513,
            0.65857445,
            0.84819673,
            0.58170179,
            0.69794437,
            0.59050779,
            0.70058127,
            0.70609522,
            0.55846892,
            0.565039,
            0.5112702,
            0.66490911,
            0.7436488,
            0.96810627,
            0.61390679,
            0.57990954,
            0.80803017,
            0.75279794,
            0.83922534,
            0.69025415,
            0.73341374,
            1.13128249,
            0.6273701,
            1.10470814,
            0.8811427,
            0.68066744,
            0.82460457,
            1.19276123,
            0.79495363,
            0.75816488,
            0.71172587,
            0.66488078,
            0.76262151,
            0.94552383,
            0.68534479,
            0.65572068,
            0.882748,
            0.79977056,
            0.55118294,
            0.62281235,
            0.680501,
            0.70540433,
            0.581517,
            0.55149741,
            0.56802557,
            0.62185372,
            0.78155762,
            0.54749483,
            0.76054534,
            0.8385011,
            0.62378051,
            0.74230446,
            0.73493398,
            0.81876926,
            0.51459809,
            0.62029124,
            0.75798972,
            0.53866147,
            0.64046192,
            0.8268219,
            0.7481851,
            0.75229256,
            1.03601974,
            0.81423211,
            0.71548541,
            0.69129492,
            0.95984395,
            1.04381321,
            0.89768278,
            0.73991055,
            1.08017568,
            0.69020896,
            0.9934152,
            1.11255394,
            0.70724154,
            0.58637093,
            0.74031772,
            1.04888874,
            0.61453812,
            0.77873537,
            0.71656128,
            0.94070088,
            0.94375249,
            0.22252446,
            0.98361321,
            0.91751983,
            1.0744854,
            1.24293223,
            0.85615635,
            0.81875818,
            0.84865362,
            0.99634551,
            0.73408522,
            0.60056938,
            0.66138416,
            0.8717211,
            0.93411509,
            0.8384349,
            0.64507901,
            0.84391847,
            0.70055969,
            0.693553,
            0.64098186,
            0.77094139,
            0.79203805,
            0.75229119,
            0.77200412,
            1.01436067,
            0.6174803,
            0.55039811,
            0.66634067,
            0.64086223,
            0.8058729,
            0.59276461,
            0.79939078,
            0.96430461,
            0.9886049,
            1.04618388,
            0.75490131,
            0.87536434,
            0.91295678,
            0.59862705,
            0.77651008,
            0.81549637,
            0.85606505,
            0.85743264,
            0.52906308,
            0.78502039,
            0.72047416,
            0.83148103,
            0.8567749,
            0.83903916,
            0.5414036,
            0.75104553,
            1.16300994,
            0.86623046,
            0.86033238,
            0.85729137,
            0.72689392,
            0.83598951,
            1.01395395,
            0.81737621,
            1.00476488,
            0.98711307,
            1.15221712,
            0.87915829,
            1.09211732,
            0.69262269,
            0.64829788,
            0.66544936,
            0.73981324,
            0.64690257,
            0.60756513,
            0.835771,
            0.65737996,
            0.57505033,
            0.81722017,
            0.76427069,
            0.75136683,
            0.65210236,
            0.7397027,
            0.85605421,
            1.02295738,
            0.97928188,
            0.69092995,
            0.96098338,
            0.84522085,
            0.70007117,
            0.9138741,
            0.747406,
            1.07543198,
            0.99502836,
            0.87624726,
            0.84563558,
            0.48883138,
            1.06719753,
            1.05485029,
            0.64649661,
            0.55016386,
            0.51911302,
            0.84939323,
            0.55591548,
            0.77068374,
            0.67598647,
            0.65623549,
            0.63358625,
            0.82363266,
            0.48563571,
            0.55980022,
            0.5264515,
            0.54235677,
            0.60398617,
            0.90887046,
            0.64115509,
            0.66882112,
            0.90907079,
            0.9097085,
            1.00970327,
            0.7051693,
            0.61015101,
            0.64971457,
            0.88365959,
            0.54912045,
            0.67514473,
            0.56264751,
            0.6989956,
            0.64707768,
            1.05763372,
            1.01586874,
            0.79464838,
            0.94702454,
            0.56181175,
            0.66685317,
            0.62762404,
            0.61197928,
            1.01971321,
            0.8610078,
            0.67438509,
            0.72973641,
            0.90060997,
            0.79993602,
            0.54419069,
            0.54986607,
            0.71485378,
            0.66875187,
            0.63482913,
            0.63209275,
            0.91609384,
            0.68330677,
            0.52234454,
            0.6676926,
            0.83874902,
            0.6012863,
            0.62987627,
            0.66066957,
            0.68848499,
            0.67253111,
            0.92465125,
            0.63651323,
            0.8244742,
            0.79342311,
            0.59708451,
            0.86856136,
            0.75797307,
            0.70514394,
            1.17649142,
            0.94464866,
            0.58067035,
            0.57754956,
            0.61154594,
            0.72579702,
            0.94286907,
            0.80156022,
            0.9699543,
            0.88586923,
            0.8174382,
            0.90114596,
            0.77572541,
            0.99581627,
            0.84852145,
            1.18517165,
            0.83324639,
            0.78769043,
            0.86041902,
            0.86289791,
            0.76923162,
            0.61399493,
            0.54209793,
            0.58197807,
            0.58553049,
            0.88341214,
            0.75499833,
            1.08516962,
            0.89433255,
            0.60128078,
            1.15651714,
            0.49045712,
            0.82827364,
            0.93499578,
            1.161046,
            1.0060731,
            0.78048954,
        ]
    ),
    "Ili": array(
        [
            0.49920406,
            0.54239663,
            0.70616712,
            0.67185809,
            0.68459424,
            0.79198604,
            0.90693042,
        ]
    ),
    "Exchange": array(
        [
            0.99598133,
            0.98883904,
            0.99674804,
            0.99275004,
            1.05859793,
            0.97667778,
            1.01972458,
            0.9681607,
        ]
    ),
}

In [26]:
clipped_H_values = {
    name: np.clip(values, 0.0, 1.0) for name, values in H_values.items()
}

for name, all_values in clipped_H_values.items():
    print(f"{name}: {np.mean(all_values):.3f} ± {np.std(all_values):.3f}")

print()

for name, all_values in clipped_H_values.items():
    print(f"{name}: {np.min(all_values):.3f} - {np.max(all_values):.3f}")

Weather: 0.864 ± 0.177
ETTh1: 0.985 ± 0.021
ETTh2: 0.977 ± 0.034
ETTm1: 0.983 ± 0.021
ETTm2: 0.979 ± 0.033
Electricity: 0.975 ± 0.065
Solar: 0.739 ± 0.043
Traffic: 0.745 ± 0.141
Ili: 0.686 ± 0.129
Exchange: 0.990 ± 0.011

Weather: 0.333 - 1.000
ETTh1: 0.943 - 1.000
ETTh2: 0.906 - 1.000
ETTm1: 0.945 - 1.000
ETTm2: 0.908 - 1.000
Electricity: 0.555 - 1.000
Solar: 0.631 - 0.822
Traffic: 0.162 - 1.000
Ili: 0.499 - 0.907
Exchange: 0.968 - 1.000
